In [148]:
import os, sys, numpy as np, pandas as pd, tensorflow as tf, re, codecs, json, time
import pickle, collections, random, math, numbers, scipy.sparse as sp, itertools, shutil,pymysql
import datetime,zipfile
import timeit
start = timeit.default_timer()
# 多核
from joblib import Parallel, delayed
import multiprocessing
#
import tensorflow as tf
def reload(mName):
    import importlib
    if mName in sys.modules:
        del sys.modules[mName]
    return importlib.import_module(mName)

from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from collections import OrderedDict, Counter
utils = reload('utils.utils')
from LotteryMFDNN import ModelMfDNN

In [335]:
class unpackFiles:
    
    def __init__(self,json_path,model_path):
        self.json_path = json_path
        self.model_path = model_path
        
    def unzip(self,zip_path,target_path):
        zip_ref = zipfile.ZipFile(zip_path,'r')
        zip_ref.extractall(target_path)
        zip_ref.close()
    
    def readJson(path):
        with open(path) as f :
            data = json.load(f)
        return data 
    
    def run(self):
        self.unzip(self.json_path,"./json")
        
        gameJson = readJson("./json/game.json")
        userLabelJson = readJson("./json/userLabel.json")
        return gameJson,userLabelJson
    
class preprocData:
    
    def __init__(self,gameJson,userLabelJson):
        """
        initial game_dict and user_dict
        """

        self.gameJson = gameJson
        self.userLabelJson = userLabelJson
        self.numGameKeys = len(gameJson.keys())-1
        self.gameDict = OrderedDict()
        
        
        self.gameTypeDict = {}
        self.gameDict['GameType_Key'] = np.asanyarray([],dtype=np.int)
        self.gameDict['genres'] = []
        self.gameDict['genres_len'] = np.asanyarray([],dtype=np.int)
        self.gameDict['daily_count'] = np.asanyarray([],dtype=np.float)
        self.gameDict['avg_rating'] = np.asanyarray([],dtype=np.float)
        self.gameDict['candidate_game_id'] = np.asanyarray([],dtype=np.int)
        self.gameDict['web_type_0'] = np.asanyarray([],dtype=np.float)
        self.gameDict['web_type_7'] = np.asanyarray([],dtype=np.float)
        
        self.userDict = OrderedDict()
        self.userDict['hall_id_trans'] = np.asanyarray([],dtype=np.int)
        self.userDict['ag_id_trans'] = np.asanyarray([],dtype=np.int)
        self.userDict['user_id_trans'] = np.asanyarray([],dtype=np.int)
        self.userDict['query_game_ids_len'] = np.asanyarray([],dtype=np.int)
        self.userDict['query_game_ids'] = []
        
        
        self.userData = pd.read_csv("./Lottery_User_Feature.csv")
        self.userData = self.userData[self.userData.columns[:-1]]
        self.gp = self.userData.groupby(['Hall_Key','AG_Key','User_Key'])
        
    def labelTransform(self,input_array,label_dict):
        label_array = input_array.copy()
        for i,j in enumerate(input_array):
            label_array[i] = label_dict[j]
        return label_array
    
    def run(self):
        
        for i in range(self.numGameKeys):
            self.gameDict['GameType_Key'] = np.append(self.gameDict['GameType_Key'],np.asanyarray([int(self.gameJson[str(i)]['gameType'])]))
            self.gameDict['genres'].append(np.asarray(self.gameJson[str(i)]['genres']))
            self.gameDict['genres_len'] = np.append(self.gameDict['genres_len'],np.asanyarray([int(self.gameJson[str(i)]['genresLength'])]))
            self.gameDict['daily_count'] = np.append(self.gameDict['daily_count'],np.asanyarray([self.gameJson[str(i)]['dailyCount']]))
            self.gameDict['avg_rating'] = np.append(self.gameDict['avg_rating'],np.asanyarray([self.gameJson[str(i)]['avgRating']]))
            self.gameDict['candidate_game_id'] = np.append(self.gameDict['candidate_game_id'],np.asanyarray([self.gameJson[str(i)]['candidateGameId']]))
            self.gameDict['web_type_0'] = np.append(self.gameDict['web_type_0'],np.asanyarray([self.gameJson[str(i)]['webType0']]))
            self.gameDict['web_type_7'] = np.append(self.gameDict['web_type_7'],np.asanyarray([self.gameJson[str(i)]['webType7']]))
            self.gameTypeDict[self.gameJson[str(i)]['gameType']] = i
            
        for i,j in enumerate(list(self.gp.groups.keys())):
            self.userDict['hall_id_trans'] = np.append(self.userDict['hall_id_trans'],self.userLabelJson['hallId'][str(j[0])])
            self.userDict['ag_id_trans'] = np.append(self.userDict['ag_id_trans'],self.userLabelJson['agId'][str(j[1])])
            self.userDict['user_id_trans'] = np.append(self.userDict['user_id_trans'],j[2])
            self.userDict['query_game_ids'].append(self.labelTransform(self.userData[self.userData.User_Key==j[2]]['GameType_Key'].unique(),self.gameTypeDict))
            self.userDict['query_game_ids_len'] = np.append(self.userDict['query_game_ids_len'],self.userData[self.userData.User_Key==j[2]]['GameType_Key'].nunique())
        
        pad = tf.keras.preprocessing.sequence.pad_sequences
        self.userDict['query_game_ids'] = list(pad(self.userDict['query_game_ids'],padding="post",maxlen=self.userDict['query_game_ids_len'].max()))



In [338]:
files = unpackFiles("./json_file/json-20190129.zip","x")
gameJson,userLabelJson = files.run()
mPreprocData = preprocData(gameJson,userLabelJson)
mPreprocData.run()

In [342]:
mPreprocData.userDict

OrderedDict([('hall_id_trans',
              array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            

In [ ]:
unzip(zip_path,target_path)
unzip("./json_file/json-20190129.zip","./json")

In [151]:
def readJson(path):
    with open(path) as f :
        data = json.load(f)
    return data

In [152]:
gameJson = readJson("./json/game.json")
userLabelJson = readJson("./json/userLabel.json")

In [327]:
numGameKeys = len(gameJson.keys())-1
gameDict = OrderedDict() 
gameTypeDict = {}
gameDict['GameType_Key'] = np.asanyarray([],dtype=np.int)
gameDict['genres'] = []
gameDict['genres_len'] = np.asanyarray([],dtype=np.int)
gameDict['daily_count'] = np.asanyarray([],dtype=np.float)
gameDict['avg_rating'] = np.asanyarray([],dtype=np.float)
gameDict['candidate_game_id'] = np.asanyarray([],dtype=np.int)
gameDict['web_type_0'] = np.asanyarray([],dtype=np.float)
gameDict['web_type_7'] = np.asanyarray([],dtype=np.float)

for i in range(numGameKeys):
    gameDict['GameType_Key'] = np.append(gameDict['GameType_Key'],np.asanyarray([int(gameJson[str(i)]['gameType'])]))
    gameDict['genres'].append(np.asarray(gameJson[str(i)]['genres']))
    gameDict['genres_len'] = np.append(gameDict['genres_len'],np.asanyarray([int(gameJson[str(i)]['genresLength'])]))
    gameDict['daily_count'] = np.append(gameDict['daily_count'],np.asanyarray([gameJson[str(i)]['dailyCount']]))
    gameDict['avg_rating'] = np.append(gameDict['avg_rating'],np.asanyarray([gameJson[str(i)]['avgRating']]))
    gameDict['candidate_game_id'] = np.append(gameDict['candidate_game_id'],np.asanyarray([gameJson[str(i)]['candidateGameId']]))
    gameDict['web_type_0'] = np.append(gameDict['web_type_0'],np.asanyarray([gameJson[str(i)]['webType0']]))
    gameDict['web_type_7'] = np.append(gameDict['web_type_7'],np.asanyarray([gameJson[str(i)]['webType7']]))
    gameTypeDict[gameJson[str(i)]['gameType']] = i
    

In [164]:
def labelTransform(input_array,label_dict):
    label_array = input_array.copy()
    for i,j in enumerate(input_array):
        label_array[i] = label_dict[j]
    return label_array

In [200]:
userData = pd.read_csv("./Lottery_User_Feature.csv")
userData = userData[userData.columns[:-1]]
#userData = userData[userData.GameType_Key!=1270]

In [227]:
gp = userData.groupby(['Hall_Key','AG_Key','User_Key'])
#np.asarray(list(gp.groups.values())[3])

In [296]:
userDict = OrderedDict()
userDict['hall_id_trans'] = np.asanyarray([],dtype=np.int)
userDict['ag_id_trans'] = np.asanyarray([],dtype=np.int)
userDict['user_id_trans'] = np.asanyarray([],dtype=np.int)
userDict['query_game_ids_len'] = np.asanyarray([],dtype=np.int)
userDict['query_game_ids'] = []
for i,j in enumerate(list(gp.groups.keys())):
    userDict['hall_id_trans'] = np.append(userDict['hall_id_trans'],userLabelJson['hallId'][str(j[0])])
    userDict['ag_id_trans'] = np.append(userDict['ag_id_trans'],userLabelJson['agId'][str(j[1])])
    userDict['user_id_trans'] = np.append(userDict['user_id_trans'],j[2])
    userDict['query_game_ids'].append(labelTransform(userData[userData.User_Key==j[2]]['GameType_Key'].unique(),gameTypeDict))
    userDict['query_game_ids_len'] = np.append(userDict['query_game_ids_len'],userData[userData.User_Key==j[2]]['GameType_Key'].nunique())

In [298]:
pad = tf.keras.preprocessing.sequence.pad_sequences
userDict['query_game_ids'] = list(pad(userDict['query_game_ids'],padding="post",maxlen=userDict['query_game_ids_len'].max()))

In [300]:
genres_list = []
max_genres = 0
for i in range(numGameKeys): 
    genres_list.append(gameJson[str(i)]['genres'])
    if max(gameJson[str(i)]['genres']) > max_genres:
        max_genres = max(gameJson[str(i)]['genres'])


In [301]:
zip_path = "./python_model/python-lottery-mfdnn-model-20190129.zip"
target_path = "./model"
def unzip(zip_path,target_path):
    zip_ref = zipfile.ZipFile(zip_path,'r')
    zip_ref.extractall(target_path)
    zip_ref.close()
    
learning_rate = 0.01
dim = 32
n_batch = 128
#modelDir = "./model-3.5.0/model_mf_with_dnn"
    
tf.reset_default_graph()
model = ModelMfDNN(
            n_items = numGameKeys,
            n_genres = max_genres+1 ,
            n_hall = len(userLabelJson['hallId']),
            n_ag = len(userLabelJson['agId']),
            max_n_genres = len(genres_list[0]),
            dim = dim,
            reg = 0.1,
            dt = datetime.datetime.now(),
            learning_rate=learning_rate)

In [302]:
len(genres_list[0])

7

In [303]:
tf.reset_default_graph()

with tf.Session(graph=model.graph) as sess:
  # Restore variables from disk.
  model.ckpt(sess, "./model/model_mf_with_dnn")
  print("Model restored.")
  predMat = []
  #for u_data, items in user_item_data(teProcessed, np.arange(nUsers), tr_item_data, n_batch=128):
  predMat.append(model.predict(sess, userDict, gameDict))
  # predMat.append(model.predict(sess, u_data))
  predMat = np.vstack(predMat)

INFO:tensorflow:Restoring parameters from ./model/model_mf_with_dnn/model-1
Model restored.


In [318]:
predDataFrame = pd.DataFrame(predMat)
predDataFrame = pd.DataFrame(predDataFrame.stack()).reset_index()

predDataFrame.columns = ['user_id_trans','game_id_trans','predRating']
predDataFrame['User_Key'] = predDataFrame['user_id_trans'].apply(lambda x : list(gp.groups.keys())[x][2] )
predDataFrame.head()

,user_id_trans,game_id_trans,predRating,User_Key
0,0,0,0.774207,86803
1,0,1,0.235786,86803
2,0,2,0.071456,86803
3,0,3,0.173086,86803
4,0,4,0.188824,86803
